# Model Downloader

This notebook just launches the downloader UI (implemented in `model_downloader_app.py`).

If you move folders around, update the path in the code cell below.


In [ ]:
%run /workspace/notebooks/model_downloader_app.py
